In [5]:
import json
import os
import glob
import numpy as np
base_dir = "data/replica_room0"
def replica2json(base_dir):
    image_path = glob.glob(os.path.join(base_dir, 'results', 'frame*.jpg'))
    image_path.sort()
    depth_path = glob.glob(os.path.join(base_dir, 'results', 'depth*.png'))
    depth_path.sort()
    intrinsic = {
        "w": 1200,
        "h": 680,
        "fx": 600.0,
        "fy": 600.0,
        "cx": 599.5,
        "cy": 339.5,
        "depth_scale": 6553.5
    }
 
    OUT = {
        "fl_x": intrinsic['fx'],
        "fl_y": intrinsic['fy'],
        "k1": 0.0,
        "k2": 0.0,
        "p1": 0.0,
        "p2": 0.0,
        "cx": intrinsic['cx'],
        "cy": intrinsic['cy'],
        "w": intrinsic['w'],
        "h": intrinsic['h'],
        "depth_scale_factor": 6553.5,
        "aabb_scale": 1,
        "frames": [],
    }
        
    with open(os.path.join(base_dir, 'traj.txt')) as f:
        lines = f.readlines()
    
    matrix_poses = []
    for i in range(0,len(image_path),10):
        line = lines[i]
        c2w = np.array(list(map(float, line.split()))).reshape(4, 4)
        c2w[:3, 1] *= -1
        c2w[:3, 2] *= -1
        matrix_poses.append(c2w)

    ## write to json
    for i, (image_name, depth_name) in enumerate(zip(image_path[::10], depth_path[::10])):
        frame = dict()
        frame['image_file_path'] = '/'.join(image_name.split('/')[-2:])
        frame['depth_file_path'] = '/'.join(depth_name.split('/')[-2:])
        frame['transform_matrix'] = matrix_poses[i].tolist()
        OUT['frames'].append(frame)
        
    with open(os.path.join(base_dir, 'transforms.json'), 'w') as f:
        json.dump(OUT, f, indent=4)  
replica2json(base_dir)